In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

Data from 2021 to validate the models

In [3]:
df1 = pd.read_sas("LLCP2021.XPT")  
print(df1.shape)
#df.head(5)

(438693, 303)


In [5]:

df_new = df1.copy()
# filter 6 features: HighBP, GenHlth, BMI, Age, MentHlth, Education and 1 target.
df_filter = df_new[['DIABETE4','_BMI5', '_AGEG5YR', 'MENTHLTH', 'GENHLTH', 'EDUCA', '_RFHYPE6']]
print(df_filter.shape)
df_filter.head(5)

(438693, 7)


,DIABETE4,_BMI5,_AGEG5YR,MENTHLTH,GENHLTH,EDUCA,_RFHYPE6
0,3.0,1454.0,11.0,10.0,5.0,4.0,1.0
1,1.0,NaN,10.0,88.0,3.0,6.0,2.0
2,1.0,2829.0,11.0,88.0,2.0,4.0,2.0
3,1.0,3347.0,9.0,10.0,2.0,4.0,2.0
4,1.0,2873.0,12.0,88.0,5.0,3.0,1.0


In [6]:
df_filter.describe()

,DIABETE4,_BMI5,_AGEG5YR,MENTHLTH,GENHLTH,EDUCA,_RFHYPE6
count,438690.000000,391841.000000,438693.000000,438691.000000,438689.000000,438688.000000,438693.000000
mean,2.761946,2855.226495,7.726016,59.923347,2.524761,5.035267,1.427244
std,0.743411,655.194977,3.645926,37.472680,1.082066,1.047852,0.699127
min,1.000000,1200.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,3.000000,2414.000000,5.000000,15.000000,2.000000,4.000000,1.000000
50%,3.000000,2744.000000,8.000000,88.000000,2.000000,5.000000,1.000000
75%,3.000000,3174.000000,11.000000,88.000000,3.000000,6.000000,2.000000
max,9.000000,9933.000000,14.000000,99.000000,9.000000,9.000000,9.000000


There are odd max values, need to adjust so that it fits the previous cleaned data.

In [7]:
df_selected = df_filter.dropna()
df_selected.shape

(391838, 7)

In [8]:
df_selected['DIABETE4'] = df_selected['DIABETE4'].replace({2:1, 3:0, 4:1})
df_selected = df_selected[df_selected.DIABETE4 != 7]
df_selected = df_selected[df_selected.DIABETE4 != 9]
df_selected.DIABETE4.unique()

array([0., 1.])

In [9]:
# _RFHYPE6
#Change 1 to 0 so it represetnts No high blood pressure and 2 to 1 so it represents high blood pressure
df_selected['_RFHYPE6'] = df_selected['_RFHYPE6'].replace({1:0, 2:1})
df_selected = df_selected[df_selected._RFHYPE6 != 9]
df_selected._RFHYPE6.unique()

array([0., 1.])

In [10]:
#_BMI5 (no changes, just note that these are BMI * 100. So for example a BMI of 4018 is really 40.18)
df_selected['_BMI5'] = df_selected['_BMI5'].div(100).round(0)
df_selected._BMI5.unique()

array([15., 28., 33., 29., 24., 46., 23., 40., 27., 35., 18., 30., 25.,
       36., 22., 21., 44., 38., 31., 45., 26., 37., 14., 32., 20., 19.,
       34., 41., 43., 39., 16., 42., 50., 51., 17., 52., 47., 49., 56.,
       57., 48., 58., 61., 53., 63., 64., 54., 55., 68., 62., 13., 59.,
       89., 66., 77., 60., 87., 69., 72., 75., 67., 79., 71., 65., 94.,
       82., 86., 12., 70., 78., 74., 98., 73., 84., 76., 83., 80., 97.,
       99., 88., 81., 92., 85., 90., 95., 91.])

In [11]:
# GENHLTH
# Remove 7 and 9 for don't know and refused
df_selected = df_selected[df_selected.GENHLTH != 7]
df_selected = df_selected[df_selected.GENHLTH != 9]
df_selected.GENHLTH.unique()

array([5., 2., 3., 4., 1.])

In [12]:
# _AGEG5YR
# already ordinal. 1 is 18-24 all the way up to 13 wis 80 and older. 5 year increments.
# remove 14 because it is don't know or missing
df_selected = df_selected[df_selected._AGEG5YR != 14]
df_selected._AGEG5YR.unique()

array([11.,  9., 12., 13., 10.,  7.,  6.,  1.,  3.,  8.,  4.,  5.,  2.])

In [13]:
# MENTHLTH = Number of Days Mental Health is Not Good
# It is already in days, scale will be between 0-30
# Change 88 to 0, means none (no bad mental health days)
# Remove all 77 Which mean: dont knows.
# Remove all 99 , which indicates interviewee refused to answer.
df_selected['MENTHLTH'] = df_selected['MENTHLTH'].replace({88:0})
df_selected = df_selected[df_selected.MENTHLTH != 77]
df_selected = df_selected[df_selected.MENTHLTH != 99]
df_selected.MENTHLTH.unique()

array([10.,  0.,  5., 25., 15., 14.,  2.,  3.,  7.,  1., 30., 20.,  8.,
        4., 21., 28., 24., 12.,  6., 22., 27.,  9., 18., 13., 17., 16.,
       19., 29., 23., 26., 11.])

In [14]:
# EDUCA = Education Level
# Remove all 9, which indicate who refused to answer.
df_selected = df_selected[df_selected.EDUCA != 9]
df_selected.EDUCA.unique()

array([4., 3., 5., 6., 2., 1.])

In [15]:
# renaming to match the column in modelling.
df_selected.rename(columns = {'_RFHYPE6':'HighBP', '_BMI5':'BMI', '_AGEG5YR': 'Age', 'TOLDHI3':'HighChol', 'DIABETE4': 'Diabetes_binary', 'GENHLTH':'GenHlth', 'MENTHLTH':'MentHlth', 'EDUCA':'Education'}, inplace = True)
df_selected.head(5)

,Diabetes_binary,BMI,Age,MentHlth,GenHlth,Education,HighBP
0,0.0,15.0,11.0,10.0,5.0,4.0,0.0
2,1.0,28.0,11.0,0.0,2.0,4.0,1.0
3,1.0,33.0,9.0,10.0,2.0,4.0,1.0
4,1.0,29.0,12.0,0.0,5.0,3.0,0.0
5,0.0,24.0,13.0,0.0,3.0,5.0,0.0


In [16]:
 # Checking for any odd values
df_selected.describe()

,Diabetes_binary,BMI,Age,MentHlth,GenHlth,Education,HighBP
count,377857.000000,377857.000000,377857.000000,377857.000000,377857.000000,377857.000000,377857.000000
mean,0.163422,28.558251,7.606994,4.156607,2.501184,5.030909,0.397050
std,0.369750,6.559507,3.557772,8.183272,1.048337,0.990007,0.489287
min,0.000000,12.000000,1.000000,0.000000,1.000000,1.000000,0.000000
25%,0.000000,24.000000,5.000000,0.000000,2.000000,4.000000,0.000000
50%,0.000000,27.000000,8.000000,0.000000,2.000000,5.000000,0.000000
75%,0.000000,32.000000,11.000000,4.000000,3.000000,6.000000,1.000000
max,1.000000,99.000000,13.000000,30.000000,5.000000,6.000000,1.000000


In [17]:
from sklearn.preprocessing import normalize

bmi_c=df_selected['BMI']
norm_bmi= normalize(bmi_c.values.reshape(1,-1)) # column into 2D array. -1 calculates the number of columns based on the original array.

# Changing BMI column into normalized values

df_selected.loc[:,'BMI']= norm_bmi[0] # replace with first row of 2D array.

In [18]:
df_selected.shape

(377857, 7)

Exporting 2021 data to use as additional test in 'FE_Model'

In [19]:
df_selected.to_csv('2021_CDC_test.csv',index=False)